---
### 1. 데이터 불러오기
- focus on 시제 (과거, 현재, 미래)
- 뉴스기사라 PyKoSpacing / Py-Hanspell no need
- 한글은 형태소 분석 필수 -> 조사 등 tagging된 데이터(문장길이) ?
- null doesnt exist

In [1]:
import pandas as pd
import numpy as np
import os
import random
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

os.listdir('./ML_project')

['cleandata', '.DS_Store', 'test.csv', 'train.csv', 'sample_submission.csv']

In [2]:
train_data = pd.read_csv('./ML_project/train.csv').drop(['ID', '유형', '극성', '확실성', 'label'], axis=1)
test_data = pd.read_csv('./ML_project/test.csv')

train_data.head()

,문장,시제
0,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,현재
1,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,과거
2,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,미래
3,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",과거
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,현재


In [3]:
train_data.shape, test_data.shape

((16541, 2), (7090, 2))

In [4]:
# train, val 분할
sp_train, sp_val,_, _ = train_test_split(train_data, train_data['시제'], test_size=0.2, random_state=13)

In [5]:
sp_train.head()

,문장,시제
4628,한국 영화 아카데미(오스카) 출품작 봉준호 감독의 ＇기생충＇(Parasite)이 또...,과거
11474,고위 공직자 재산 신고 결과 주택 보유 수가 많은 고위 공직자 상위 10명 가운데 ...,현재
9626,판타자이는 양준일이 JTBC ＇슈가맨3＇ 출연했을 당시 7000여명의 회원들이 가입...,과거
4840,일본은 최고 45% 7단계 누진세율을 적용하고 있다.,현재
16281,그러면 왜 정통 의학에서는 레이어트릴을 항암제로 인정하지 않으려는 것일까,현재


In [6]:
sp_train['시제'].value_counts()

과거    6440
현재    5469
미래    1323
Name: 시제, dtype: int64

In [7]:
sp_train.shape, sp_val.shape

((13232, 2), (3309, 2))

In [8]:
# Label Encoding (시제)
le = LabelEncoder()
sp_train['시제'] = le.fit_transform(sp_train['시제'].values)
sp_val['시제'] = le.fit_transform(sp_val['시제'].values)
 
def encoding(sp_train, sp_val): 
    train_tense = sp_train["시제"].values
    val_tense = sp_val["시제"].values


    sp_train_labels={
        'tense' : train_tense
    }

    sp_val_labels = {
        'tense' : val_tense
    }

    return sp_train_labels, sp_val_labels


In [9]:
sp_train.head()

,문장,시제
4628,한국 영화 아카데미(오스카) 출품작 봉준호 감독의 ＇기생충＇(Parasite)이 또...,0
11474,고위 공직자 재산 신고 결과 주택 보유 수가 많은 고위 공직자 상위 10명 가운데 ...,2
9626,판타자이는 양준일이 JTBC ＇슈가맨3＇ 출연했을 당시 7000여명의 회원들이 가입...,0
4840,일본은 최고 45% 7단계 누진세율을 적용하고 있다.,2
16281,그러면 왜 정통 의학에서는 레이어트릴을 항암제로 인정하지 않으려는 것일까,2


In [10]:
#리뷰 전체길이 확인
train_length = sp_train['문장'].astype(str).apply(len)
train_length.head()

4628     62
11474    69
9626     50
4840     29
16281    40
Name: 문장, dtype: int64

In [11]:
#리뷰 통계 정보
print('문장 길이 최댓값: {}'.format(np.max(train_length)))
print('문장 길이 최솟값: {}'.format(np.min(train_length)))
print('문장 길이 평균값: {:.2f}'.format(np.mean(train_length)))
print('문장 길이 표준편차: {:.2f}'.format(np.std(train_length)))
print('문장 길이 중간값: {}'.format(np.median(train_length)))
print('문장 길이 제1사분위: {}'.format(np.percentile(train_length,25)))
print('문장 길이 제3사분위: {}'.format(np.percentile(train_length,75)))

문장 길이 최댓값: 534
문장 길이 최솟값: 7
문장 길이 평균값: 63.71
문장 길이 표준편차: 35.67
문장 길이 중간값: 56.0
문장 길이 제1사분위: 40.0
문장 길이 제3사분위: 79.0


In [12]:
# 문자열 아닌 데이터 모두 제거
train_str = [i for i in sp_train['문장'] if type(i) is str]
train_str

['한국 영화 아카데미(오스카) 출품작 봉준호 감독의 ＇기생충＇(Parasite)이 또 하나의 수상 소식을 알렸다.',
 '고위 공직자 재산 신고 결과 주택 보유 수가 많은 고위 공직자 상위 10명 가운데 서울시의원이 4명이나 포함돼 눈길을 끈다.',
 '판타자이는 양준일이 JTBC ＇슈가맨3＇ 출연했을 당시 7000여명의 회원들이 가입했었다.',
 '일본은 최고 45% 7단계 누진세율을 적용하고 있다.',
 '그러면 왜 정통 의학에서는 레이어트릴을 항암제로 인정하지 않으려는 것일까',
 '비둘기나 작은 병아리마저 무섭다.',
 '2013년부터는 사지 말고 고쳐 입으라는 메시지를 강조하기 위해 지퍼·단추 교체 등 DIY 수선법을 가르쳐주는 동영상 45개를 배포했다.',
 '2010년 그 시절 우리가 사랑했던 아이돌이 돌아왔다.',
 '암호화폐 거래소들에서 전자사기 범죄가 빈번하게 일어나고 있다.',
 '1일 울산 울주군 간절곶을 찾은 해맞이객들은 올해 첫 일출을 감상하면서 가족 건강과 행복 등 소원을 빌었다.',
 '좋은 백스윙을 했지만 그래도 좀 더 강하게 볼을 치고 싶은 욕심이 있겠죠',
 '크래프트에그와 부시로드가 공동 제작한 모바일 리듬게임 ＇뱅드림! 걸즈밴드파티＇ 역시 완성도 높은 게임성으로 호평 받았다.',
 '이자 비용은 가계가 지출하는 주택 담보 대출이나 신용대출, 전세자금 대출, 학자금 대출 등의 이자가 포함된 금액으로, 최근 금리 인상이 이어지며 부담이 늘어난 것으로 풀이된다.',
 '제92회 아카데미 시상식을 앞두고 전 세계 SNS는 ＇기생충＇에 관한 포스트로 뒤덮였다.',
 '한국 정부도 노인 일자리를 만들기 위해 노력해왔다.',
 '요즘 냉면의 대명사는 평양냉면이다.',
 '최강 연맹전에서 승리한 연맹은 게임 내 모든 이용자의 영지에 해당 연맹의 깃발이 표기할 수 있으며 게임 내 아이템도 받는다.',
 '당시 보고서에는 ＂심장병과 뇌졸중, 암, 당뇨병 등으로 연간 350만 명이 죽어가고 있으며, 이는 전체 사망자의 60%에 해당한다.＂라

In [13]:
print('과거 문장 갯수: {}'.format(sp_train['시제'].value_counts()[0]))
print('현재 문장 갯수: {}'.format(sp_train['시제'].value_counts()[1]))
print('미래 문장 갯수: {}'.format(sp_train['시제'].value_counts()[2]))


과거 문장 갯수: 6440
현재 문장 갯수: 1323
미래 문장 갯수: 5469


---
### 2. 전체 데이터 문장 전처리

In [14]:
import re
import json
from konlpy.tag import Okt

In [15]:
def preprocessing(sentence, t, remove_stopwords=False, stop_words=[]):

    # 1. 한글 및 공백 제외한 문자 제거
    sentence_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]','',sentence)
    # 2. 형태소 단어 나눔
    sentence_review = t.morphs(sentence_text,stem=True)

    # 3. 불용어 제거
    if remove_stopwords:
        sentence_review = [i for i in sentence_review if not i in stop_words]
    
    return sentence_review


In [16]:
# 전체 텍스트 전처리
stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한']
t = Okt()
clean_train_sentence = []

for sentence in sp_train['문장']:
    # 문자열인 경우 전처리 진행
    if type(sentence) == str:
        clean_train_sentence.append(preprocessing(sentence, t, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_train_sentence.append([]) #빈칸

clean_train_sentence[:5]

[['한국',
  '영화',
  '아카데미',
  '오스카',
  '출품',
  '작',
  '봉준호',
  '감독',
  '기생충',
  '또',
  '하나',
  '수상',
  '소식',
  '을',
  '알리다'],
 ['고위',
  '공직자',
  '재산',
  '신고',
  '결과',
  '주택',
  '보유',
  '수가',
  '많다',
  '고위',
  '공직자',
  '상위',
  '명',
  '가운데',
  '서울시',
  '의원',
  '명',
  '이나',
  '포함',
  '돼다',
  '눈길',
  '을',
  '끌다'],
 ['판', '타자', '양준', '일이', '슈가', '맨', '출연', '하다', '당시', '여명', '회원', '가입', '하다'],
 ['일본', '최고', '단계', '누진세', '율', '을', '적용', '하고', '있다'],
 ['그러면',
  '왜',
  '정통',
  '의학',
  '에서는',
  '레이어',
  '트릴',
  '을',
  '항암제',
  '로',
  '인정',
  '하다',
  '않다',
  '이다']]

In [20]:
# 테스트 데이터도 동일하게 전처리

clean_test_sentence = []

for sentence in sp_val['문장']:

  if type(sentence) == str:
    clean_test_sentence.append(preprocessing(sentence, t, remove_stopwords=True, stop_words=stop_words))
  else:
    clean_test_sentence.append([])

clean_test_sentence[:5]

[['자동차',
  '생산',
  '만으로는',
  '미래',
  '를',
  '기약',
  '하다',
  '없다',
  '항공',
  '영역',
  '으로',
  '파고들다',
  '혁신',
  '카드',
  '를',
  '던지다',
  '이다'],
 ['김씨',
  '소속사',
  '인',
  '건음',
  '기획',
  '악의',
  '적',
  '인',
  '의도',
  '라며',
  '씨',
  '를',
  '명예훼손',
  '혐의',
  '로',
  '맞고소',
  '하다'],
 ['영화',
  '년',
  '부터',
  '연재',
  '누',
  '계',
  '만',
  '부르다',
  '돌파',
  '니',
  '시',
  '모리',
  '히로유키',
  '만화',
  '오늘',
  '부터',
  '우리',
  '원작',
  '이다'],
 ['하지만', '난관', '지금', '부터', '다'],
 ['그렇다', '년', '팔자', '로', '돌아서다']]

#### str 문장 데이터 벡터로 변환

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [21]:
# 인덱스 벡터 변환 후 일정 길이 넘어가거나 모자라는 문장 패딩처리
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_sentence)
train_sequences = tokenizer.texts_to_sequences(clean_train_sentence)
test_sequences = tokenizer.texts_to_sequences(clean_test_sentence)

word_vocab = tokenizer.word_index #단어사전형태
MAX_SEQUENCE_LENGTH = 10 #문장 최대 길이

#학습 데이터
train_inputs = pad_sequences(train_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')

#학습 데이터 라벨 벡터화
train_labels = np.array(sp_train['시제'])

#평가 데이터 
test_inputs = pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH,padding='post')
#평가 데이터 라벨 벡터화
test_labels = np.array(sp_val['시제'])

In [22]:
# 전처리 한 데이터 파일로 저장
DEFAULT_PATH  = './ML_project/' # 경로지정
DATA_PATH = 'cleandata/' #.npy파일 저장 경로지정
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TEST_INPUT_DATA = 'test_input.npy'
TEST_LABEL_DATA = 'test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs={}
data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) + 1

#전처리한 데이터들 파일로저장
import os

if not os.path.exists(DEFAULT_PATH + DATA_PATH):
  os.makedirs(DEFAULT_PATH+DATA_PATH)

#전처리 학습데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_INPUT_DATA,'wb'),train_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TRAIN_LABEL_DATA,'wb'),train_labels)
#전처리 테스트데이터 넘파이로 저장
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_INPUT_DATA,'wb'),test_inputs)
np.save(open(DEFAULT_PATH+DATA_PATH+TEST_LABEL_DATA,'wb'),test_labels)

#데이터 사전 json으로 저장
json.dump(data_configs,open(DEFAULT_PATH + DATA_PATH + DATA_CONFIGS,'w'),ensure_ascii=False)


---
### 1. 데이터 불러오기

---
### 2. 파라미터 세팅

---
### 3. 모델 함수
- CNN ? 

---
### 4. 학습

---
#### 참고 

- https://wikidocs.net/92961
- https://dacon.io/competitions/official/236037/codeshare/7260?page=2&dtype=recent
- https://wikidocs.net/92961
- https://wonhwa.tistory.com/35
- https://hyemin-kim.github.io/2020/08/29/E-Python-TextMining-2/#3-0-konlpy-%EC%84%A4%EC%B9%98
- https://blog.naver.com/chensi/222894582744
- https://blog.naver.com/nabilera1/222299616157
